# **Inisialisasi**


In [ ]:
!pip install wandb
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 30.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import wandb
import os

os.environ["WANDB_PROJECT"] = "BERT"
os.environ["WANDB_LOG_MODEL"] = "true"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


::

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import numpy as np
import pandas as pd
import random


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
# from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer

# **Preprocesing**

In [ ]:
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = 'intents.json'

intents = load_json_file(filename)

In [ ]:
def create_df():
    df = pd.DataFrame({
        'Pattern' : [],
        'Tag' : []
    })

    return df

df = create_df()
df

,Pattern,Tag


In [ ]:
def extract_json_info(json_file, df):

    for intent in json_file['intents']:

        for pattern in intent['patterns']:

            sentence_tag = [pattern, intent['tag']]
            df.loc[len(df.index)] = sentence_tag

    return df

df = extract_json_info(intents, df)
df.head()

,Pattern,Tag
0,Apa itu gempa bumi?,pengertian_gempa
1,Jelaskan gempa bumi,pengertian_gempa
2,Pengertian dari gempa bumi,pengertian_gempa
3,Gempa bumi itu apa?,pengertian_gempa
4,Pengertian gempa bumi,pengertian_gempa


In [ ]:
df2 = df.copy()
df2.head()

,Pattern,Tag
0,Apa itu gempa bumi?,pengertian_gempa
1,Jelaskan gempa bumi,pengertian_gempa
2,Pengertian dari gempa bumi,pengertian_gempa
3,Gempa bumi itu apa?,pengertian_gempa
4,Pengertian gempa bumi,pengertian_gempa


In [ ]:
labels = df2['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels

['pengertian_gempa',
 'penyebab_gempa',
 'jenis_gempa',
 'gempa_tektonik',
 'gempa_vulkanik',
 'gempa_dampak_meteorit',
 'gempa_buatan',
 'skala_gempa',
 'alat_ukur_gempa',
 'richter',
 'magnitudo',
 'tingkatan_skala_magnitudo',
 'perbedaan_skala_ritcher_dan_magnitudo',
 'hiposentrum_dan_episentrum',
 'dampak_gempa',
 'seismik',
 'seismologi',
 'P_wave',
 'S_wave',
 'tsunami',
 'tsunami_akibat_gempa',
 'tsunami_terparah_indonesia',
 'bisakah_prediksi_gempa',
 'bisakah_prediksi_tsunami',
 'hewan_prediksi_gempa',
 'wilayah_rawan_gempa',
 'patahan',
 'patahan_naik',
 'patahan_normal',
 'patahan_geser',
 'patahan_divergen',
 'patahan_terbalik',
 'patahan_miring',
 'gempa_akibat_patahan',
 'lempeng',
 'lapisan_bumi',
 'kerak_bumi',
 'jenis_kerak',
 'mantel',
 'inti',
 'mitigasi',
 'persiapan',
 'rencana_tanggap',
 'siap_dokumen',
 'mitigasi_dalam_gedung',
 'mitigasi_dalam_rumah',
 'mitigasi_dalam_lift',
 'mitigasi_luar_ruangan',
 'mitigasi_tempat_ramai',
 'tempat_paling_aman',
 'setelah_gem

In [ ]:
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}

In [ ]:
id2label

{0: 'pengertian_gempa',
 1: 'penyebab_gempa',
 2: 'jenis_gempa',
 3: 'gempa_tektonik',
 4: 'gempa_vulkanik',
 5: 'gempa_dampak_meteorit',
 6: 'gempa_buatan',
 7: 'skala_gempa',
 8: 'alat_ukur_gempa',
 9: 'richter',
 10: 'magnitudo',
 11: 'tingkatan_skala_magnitudo',
 12: 'perbedaan_skala_ritcher_dan_magnitudo',
 13: 'hiposentrum_dan_episentrum',
 14: 'dampak_gempa',
 15: 'seismik',
 16: 'seismologi',
 17: 'P_wave',
 18: 'S_wave',
 19: 'tsunami',
 20: 'tsunami_akibat_gempa',
 21: 'tsunami_terparah_indonesia',
 22: 'bisakah_prediksi_gempa',
 23: 'bisakah_prediksi_tsunami',
 24: 'hewan_prediksi_gempa',
 25: 'wilayah_rawan_gempa',
 26: 'patahan',
 27: 'patahan_naik',
 28: 'patahan_normal',
 29: 'patahan_geser',
 30: 'patahan_divergen',
 31: 'patahan_terbalik',
 32: 'patahan_miring',
 33: 'gempa_akibat_patahan',
 34: 'lempeng',
 35: 'lapisan_bumi',
 36: 'kerak_bumi',
 37: 'jenis_kerak',
 38: 'mantel',
 39: 'inti',
 40: 'mitigasi',
 41: 'persiapan',
 42: 'rencana_tanggap',
 43: 'siap_dokumen

In [ ]:
label2id


{'pengertian_gempa': 0,
 'penyebab_gempa': 1,
 'jenis_gempa': 2,
 'gempa_tektonik': 3,
 'gempa_vulkanik': 4,
 'gempa_dampak_meteorit': 5,
 'gempa_buatan': 6,
 'skala_gempa': 7,
 'alat_ukur_gempa': 8,
 'richter': 9,
 'magnitudo': 10,
 'tingkatan_skala_magnitudo': 11,
 'perbedaan_skala_ritcher_dan_magnitudo': 12,
 'hiposentrum_dan_episentrum': 13,
 'dampak_gempa': 14,
 'seismik': 15,
 'seismologi': 16,
 'P_wave': 17,
 'S_wave': 18,
 'tsunami': 19,
 'tsunami_akibat_gempa': 20,
 'tsunami_terparah_indonesia': 21,
 'bisakah_prediksi_gempa': 22,
 'bisakah_prediksi_tsunami': 23,
 'hewan_prediksi_gempa': 24,
 'wilayah_rawan_gempa': 25,
 'patahan': 26,
 'patahan_naik': 27,
 'patahan_normal': 28,
 'patahan_geser': 29,
 'patahan_divergen': 30,
 'patahan_terbalik': 31,
 'patahan_miring': 32,
 'gempa_akibat_patahan': 33,
 'lempeng': 34,
 'lapisan_bumi': 35,
 'kerak_bumi': 36,
 'jenis_kerak': 37,
 'mantel': 38,
 'inti': 39,
 'mitigasi': 40,
 'persiapan': 41,
 'rencana_tanggap': 42,
 'siap_dokumen': 4

In [ ]:
df2['labels'] = df2['Tag'].map(lambda x: label2id[x.strip()])
df2.head()

,Pattern,Tag,labels
0,Apa itu gempa bumi?,pengertian_gempa,0
1,Jelaskan gempa bumi,pengertian_gempa,0
2,Pengertian dari gempa bumi,pengertian_gempa,0
3,Gempa bumi itu apa?,pengertian_gempa,0
4,Pengertian gempa bumi,pengertian_gempa,0


In [ ]:
X = list(df2['Pattern'])
X[:5]

['Apa itu gempa bumi?',
 'Jelaskan gempa bumi',
 'Pengertian dari gempa bumi',
 'Gempa bumi itu apa?',
 'Pengertian gempa bumi']

In [ ]:
y = list(df2['labels'])
y[:5]

[0, 0, 0, 0, 0]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 123)



In [ ]:
model_name = "bert-base-uncased"
max_len = 1024

tokenizer = BertTokenizer.from_pretrained(model_name,
                                          max_length=max_len)

model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=num_labels,
                                                      id2label=id2label,
                                                      label2id = label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
full_data = tokenizer(X, truncation=True, padding=True)



In [ ]:
class DataLoader(Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)


In [ ]:
fullDataLoader = DataLoader(full_data, y_test)

# **Finetuning**

In [ ]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Massive/output',
    do_train=True,
    do_eval=True,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=10,
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to="wandb"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=test_dataloader,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,4.570200,4.360112,0.034091,0.006034,0.004900,0.029514
100,3.924800,3.614377,0.291667,0.202102,0.209414,0.273264
150,3.111000,2.878671,0.545455,0.421424,0.412895,0.513021
200,2.371800,2.300424,0.636364,0.522862,0.529903,0.589089
250,1.759300,1.783617,0.757576,0.678333,0.677369,0.738628
300,1.279000,1.395919,0.803030,0.737232,0.742188,0.783594
350,0.902000,1.121115,0.833333,0.774192,0.775090,0.818814
400,0.631600,0.872478,0.852273,0.808074,0.814385,0.844705
450,0.438100,0.723009,0.890152,0.861608,0.874888,0.884635
500,0.294700,0.592573,0.909091,0.894571,0.919975,0.907456


TrainOutput(global_step=1250, training_loss=0.8111111004829407, metrics={'train_runtime': 381.1515, 'train_samples_per_second': 103.633, 'train_steps_per_second': 3.28, 'total_flos': 893940612840000.0, 'train_loss': 0.8111111004829407, 'epoch': 50.0})

# **Evaluation**

In [ ]:
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]

pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.027204,1.000000,1.00000,1.000000,1.00000
test,0.386129,0.920455,0.90935,0.929975,0.92307


In [ ]:
def predict(text):

    inputs = tokenizer(text, padding=True, truncation=True, max_length=1024, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [ ]:
text = "jelaskn apa itu gempa bumi"
predict(text)

(tensor([[9.7851e-01, 1.0718e-03, 1.2963e-03, 2.0807e-04, 1.3590e-04, 2.5264e-04,
          1.5357e-04, 8.7071e-04, 7.8776e-04, 2.3288e-04, 6.2618e-05, 3.3014e-04,
          1.2696e-04, 9.0920e-05, 1.4977e-03, 4.2780e-04, 8.9604e-05, 2.4298e-04,
          3.2084e-04, 1.2910e-04, 5.3907e-04, 1.6574e-04, 1.2985e-04, 8.1199e-05,
          1.3609e-04, 2.1598e-05, 8.4915e-05, 9.5958e-05, 8.1293e-05, 9.7028e-06,
          7.6144e-06, 5.1003e-05, 1.8304e-05, 4.8533e-05, 4.3849e-06, 3.1660e-04,
          1.6405e-03, 4.4905e-04, 4.9223e-05, 6.8460e-05, 3.6319e-04, 3.1463e-04,
          1.2747e-04, 4.5438e-05, 8.8654e-05, 9.9053e-05, 1.1642e-04, 3.2356e-05,
          4.8086e-05, 3.1406e-05, 5.4792e-05, 3.1490e-04, 1.5381e-05, 9.2773e-05,
          5.9003e-05, 6.5446e-05, 5.3932e-05, 5.4948e-05, 8.8845e-04, 6.2414e-04,
          1.1448e-03, 3.3587e-04, 3.4384e-05, 4.5050e-05, 2.7313e-05, 5.9506e-05,
          1.2967e-04, 7.8482e-05, 4.8671e-04, 6.8192e-05, 1.0526e-04, 4.7039e-05,
          4.7153

In [ ]:
model_path = "/content/drive/MyDrive/Massive/BERT"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Massive/BERT/tokenizer_config.json',
 '/content/drive/MyDrive/Massive/BERT/special_tokens_map.json',
 '/content/drive/MyDrive/Massive/BERT/vocab.txt',
 '/content/drive/MyDrive/Massive/BERT/added_tokens.json')

Testing

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
# from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer
model_path = "/content/drive/MyDrive/Massive/BERT"


model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
chatbot= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
chatbot("gempa terbesar di sejarah")

[{'label': 'gempa_terbesar_dunia', 'score': 0.9584900140762329}]

In [ ]:
def chat(chatbot):

    print("Chatbot: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..")
    print("Type 'quit' to exit the chat\n\n")

    text = input("User: ").strip().lower()

    while(text != 'quit'):

        score = chatbot(text)[0]['score']

        if score < 0.8:
            print("Chatbot: Sorry I can't answer that\n\n")
            text = input("User: ").strip().lower()
            continue

        label = label2id[chatbot(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])

        print(f"Chatbot: {response}\n\n")

        text = input("User: ").strip().lower()

In [ ]:
chat(chatbot)

Chatbot: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..
Type 'quit' to exit the chat


User: pENGERTIAN GEMPA BUMI
Chatbot: Gempa bumi adalah getaran atau getar-getar yang terjadi di permukaan bumi akibat pelepasan energi dari dalam secara tiba-tiba yang menciptakan gelombang seismik.


User: apa sih gempa bumi
Chatbot: Gempa bumi adalah getaran atau getar-getar yang terjadi di permukaan bumi akibat pelepasan energi dari dalam secara tiba-tiba yang menciptakan gelombang seismik.


User: gempa akibat patahan
Chatbot: Ya, patahan dapat menyebabkan gempa bumi. Patahan adalah retakan di kerak bumi yang terjadi ketika dua lempeng bumi bergerak relatif terhadap satu sama lain. Ketika gaya tekan kerak bumi mencapai ambang ketegangan, akumulasi energi potensial di dispersikan oleh pelepasan tegangan, yang dapat menyebabkan gempa bumi.


User: quit
